## Assignment 1a)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
dt = 0.1  # Time step (ms)
T = 1000  # Total simulation time (ms)
n_steps = int(T / dt)  # Number of steps
I = 10  # Input current (arbitrary value)

# Initialize variables
v = -65  # Membrane potential (initial value)
u = 0  # Recovery variable (initial value)
v_values = []  # To store membrane potential over time
u_values = []  # To store recovery variable over time
time = np.arange(0, T, dt)  # Time vector

# Modified Euler method implementation
for t in time:
    v_values.append(v)
    u_values.append(u)
    
    # Calculate derivatives
    dv = (0.7 * (v + 60) * (v + 40) - u + I) / 100
    du = 0.03 * (-2 * (v + 60) - u)
    
    # Predictor step
    v_pred = v + dv * dt
    u_pred = u + du * dt
    
    # Corrector step
    dv_pred = (0.7 * (v_pred + 60) * (v_pred + 40) - u_pred + I) / 100
    du_pred = 0.03 * (-2 * (v_pred + 60) - u_pred)
    
    v += (dv + dv_pred) / 2 * dt
    u += (du + du_pred) / 2 * dt
    
    # Reset conditions
    if v > 35:  # Spike condition
        v = -50
        u += 100

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(time, v_values, label='Membrane Potential (v)')
plt.xlabel('Time (ms)')
plt.ylabel('Membrane Potential (mV)')
plt.title('Simulation of a Regular Spiking Pyramidal Cell')
plt.legend()
plt.grid()
plt.show()


## Assignment 1b)

### For what values does the neuron spike?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to simulate the model for a given I
def simulate(I, dt=0.1, T=1000):
    n_steps = int(T / dt)
    v = -65
    u = 0
    v_values, u_values, spikes = [], [], 0
    time = np.arange(0, T, dt)

    for t in time:
        v_values.append(v)
        u_values.append(u)

        dv = (0.7 * (v + 60) * (v + 40) - u + I) / 100
        du = 0.03 * (-2 * (v + 60) - u)

        v_pred = v + dv * dt
        u_pred = u + du * dt

        dv_pred = (0.7 * (v_pred + 60) * (v_pred + 40) - u_pred + I) / 100
        du_pred = 0.03 * (-2 * (v_pred + 60) - u_pred)

        v += (dv + dv_pred) / 2 * dt
        u += (du + du_pred) / 2 * dt

        if v > 35:
            v = -50
            u += 100
            spikes += 1

    return time, v_values, u_values, spikes

# Parameter sweep for I
I_values = np.arange(0, 20, 0.5)  # Test input currents
spike_counts = []

for I in I_values:
    _, _, _, spikes = simulate(I, T=500)
    spike_counts.append(spikes)

# Plotting spikes as a function of I
plt.figure(figsize=(8, 5))
plt.plot(I_values, spike_counts, marker='o', label="Spike Count")
plt.axvline(x=I_values[next(i for i, s in enumerate(spike_counts) if s > 0)], color='r', linestyle='--', label="Spike Threshold")
plt.xlabel('Input Current (I)')
plt.ylabel('Number of Spikes')
plt.title('Threshold Current for Spiking')
plt.legend()
plt.grid()
plt.show()


### The bifurcation at the onset of spiking

In [ ]:
def phase_plane(I, v_range=np.linspace(-70, 50, 100), u_range=np.linspace(-20, 150, 100)):
    v, u = np.meshgrid(v_range, u_range)
    dv = (0.7 * (v + 60) * (v + 40) - u + I) / 100
    du = 0.03 * (-2 * (v + 60) - u)

    plt.figure(figsize=(8, 6))
    plt.streamplot(v, u, dv, du, density=1.2, color='blue', linewidth=0.8)
    plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='black', linestyle='--', linewidth=0.5)
    plt.title(f'Phase Plane Analysis (I = {I})')
    plt.xlabel('Membrane Potential (v)')
    plt.ylabel('Recovery Variable (u)')
    plt.grid()
    plt.show()

# Test for a specific I
phase_plane(I=5)


### How does the firing rate vary with I?

In [ ]:
firing_rates = []

for I in I_values:
    _, _, _, spikes = simulate(I, T=1000)
    firing_rates.append(spikes)

# Plot firing rate vs I
plt.figure(figsize=(8, 5))
plt.plot(I_values, firing_rates, marker='o', label='Firing Rate (Hz)')
plt.xlabel('Input Current (I)')
plt.ylabel('Firing Rate (Hz)')
plt.title('Firing Rate vs Input Current')
plt.legend()
plt.grid()
plt.show()


### What type of spiking behavior does this neuron show?

Based on the dynamics, the neuron likely exhibits regular spiking behavior:
- The spiking behavior is periodic and depends on I, typical for pyramidal cells. 
- The firing rate smoothly increases with I, which is characteristic of type I spiking neurons.

## Assignment 1c)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 10  # Number of neurons
T = 1000  # Total simulation time (ms)
dt = 0.1  # Time step (ms)
n_steps = int(T / dt)
I_ext = np.random.uniform(8, 12, N)  # External input current for each neuron
W = np.random.uniform(0, 2, (N, N))  # Connection matrix (random coupling strengths)
np.fill_diagonal(W, 0)  # No self-connections

# Initialize variables
v = np.full(N, -65.0)  # Initial membrane potentials
u = np.zeros(N)  # Initial recovery variables
spike_counts = np.zeros(N)  # Count spikes for each neuron
v_values = np.zeros((N, n_steps))  # Membrane potentials over time
spike_times = [[] for _ in range(N)]  # Spike times for each neuron
time = np.arange(0, T, dt)

# Simulate network dynamics
for step in range(n_steps):
    for i in range(N):
        # Compute derivatives
        dv = (0.7 * (v[i] + 60) * (v[i] + 40) - u[i] + I_ext[i]) / 100
        du = 0.03 * (-2 * (v[i] + 60) - u[i])

        # Predictor step
        v_pred = v[i] + dv * dt
        u_pred = u[i] + du * dt

        # Corrector step
        dv_pred = (0.7 * (v_pred + 60) * (v_pred + 40) - u_pred + I_ext[i]) / 100
        du_pred = 0.03 * (-2 * (v_pred + 60) - u_pred)

        # Update variables
        v[i] += (dv + dv_pred) / 2 * dt
        u[i] += (du + du_pred) / 2 * dt

        # Check for spike and apply reset
        if v[i] > 35:
            v[i] = -50
            u[i] += 100
            spike_counts[i] += 1
            spike_times[i].append(step * dt)

            # Send current to connected neurons based on connection matrix
            I_ext += W[:, i]

    # Store membrane potentials
    v_values[:, step] = v

# Plotting the results
plt.figure(figsize=(10, 8))
for i in range(N):
    plt.plot(time, v_values[i, :], label=f'Neuron {i + 1}')
plt.xlabel('Time (ms)')
plt.ylabel('Membrane Potential (mV)')
plt.title('Network Dynamics of Coupled Neurons')
plt.legend()
plt.grid()
plt.show()

# Print spike counts
print("Spike counts for each neuron:", spike_counts)


## Assignment 1d)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 10  # Number of neurons
T = 1000  # Total simulation time (ms)
dt = 0.1  # Time step (ms)
n_steps = int(T / dt)
I_ext = np.random.uniform(8, 12, N)  # External input current for each neuron
base_W = np.random.uniform(0, 1, (N, N))  # Base connection matrix
np.fill_diagonal(base_W, 0)  # No self-connections

# Function to simulate network dynamics for a given coupling strength
def simulate_network(W_s):
    W = base_W * W_s  # Scale connection matrix by coupling strength
    v = np.full(N, -65.0)  # Initial membrane potentials
    u = np.zeros(N)  # Initial recovery variables
    spike_counts = np.zeros(N)  # Count spikes for each neuron
    v_values = np.zeros((N, n_steps))  # Membrane potentials over time
    spike_times = [[] for _ in range(N)]  # Spike times for rastergram
    time = np.arange(0, T, dt)

    for step in range(n_steps):
        for i in range(N):
            # Compute derivatives
            dv = (0.7 * (v[i] + 60) * (v[i] + 40) - u[i] + I_ext[i]) / 100
            du = 0.03 * (-2 * (v[i] + 60) - u[i])

            # Predictor step
            v_pred = v[i] + dv * dt
            u_pred = u[i] + du * dt

            # Corrector step
            dv_pred = (0.7 * (v_pred + 60) * (v_pred + 40) - u_pred + I_ext[i]) / 100
            du_pred = 0.03 * (-2 * (v_pred + 60) - u_pred)

            # Update variables
            v[i] += (dv + dv_pred) / 2 * dt
            u[i] += (du + du_pred) / 2 * dt

            # Check for spike and apply reset
            if v[i] > 35:
                v[i] = -50
                u[i] += 100
                spike_counts[i] += 1
                spike_times[i].append(step * dt)

                # Send current to connected neurons based on connection matrix
                I_ext += W[:, i]

        # Store membrane potentials
        v_values[:, step] = v

    return time, v_values, spike_times, spike_counts

# Rastergram plot function
def plot_rastergram(spike_times, title):
    plt.figure(figsize=(10, 6))
    for neuron_id, spikes in enumerate(spike_times):
        plt.scatter(spikes, [neuron_id + 1] * len(spikes), s=5)
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron Index')
    plt.title(title)
    plt.grid(True)
    plt.show()

# Coupling strengths to test
coupling_strengths = [0, 0.5, 1, 2]  # Explore weak to strong coupling
firing_rates = []

# Run simulations and analyze
for W_s in coupling_strengths:
    time, v_values, spike_times, spike_counts = simulate_network(W_s)

    # Compute mean firing rate
    mean_firing_rate = np.sum(spike_counts) / (N * (T / 1000))  # in Hz
    firing_rates.append(mean_firing_rate)

    # Plot rastergram
    plot_rastergram(
        spike_times,
        title=f'Rastergram (Coupling Strength W_s = {W_s})'
    )

# Plot firing rate vs coupling strength
plt.figure(figsize=(8, 5))
plt.plot(coupling_strengths, firing_rates, marker='o')
plt.xlabel('Coupling Strength (W_s)')
plt.ylabel('Mean Firing Rate (Hz)')
plt.title('Firing Rate vs Coupling Strength')
plt.grid(True)
plt.show()


## Assignment 2a)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters for the leaky integrate-and-fire neuron
tau = 10.0  # Membrane time constant (ms)
I = 1.2  # Input current (must be > 1 for spiking)
V_reset = 0  # Reset potential after a spike
V_threshold = 1  # Threshold potential for spike
dt = 0.01  # Time step (ms)
t_max = 100  # Total simulation time (ms)

# Step 1: Associate a phase with each value of V
# Solve the leaky integrate-and-fire equation analytically
def membrane_potential(t, tau, I):
    """Membrane potential V(t) as a function of time."""
    return 1 - np.exp(-t / tau)

# Invert the function to get phase phi as a function of V
def phase_from_voltage(V, tau, I):
    """Phase as a function of membrane potential V."""
    return -tau * np.log(1 - V)

# Step 2: Determine the change in V after a current pulse
def apply_pulse(V, delta_V):
    """Apply a small perturbation to the membrane potential."""
    return V + delta_V

# Step 3: Compute the PRC
def calculate_prc(tau, I, delta_V):
    """Calculate the phase resetting curve (PRC)."""
    # Time points to sample for the PRC
    V_values = np.linspace(0, V_threshold, 1000)[:-1]  # Avoid hitting V_threshold
    phi_values = phase_from_voltage(V_values, tau, I)
    delta_phi_values = []

    for V in V_values:
        # Apply a small perturbation delta_V at this voltage
        V_perturbed = apply_pulse(V, delta_V)

        # Check if perturbation exceeds the threshold
        if V_perturbed >= V_threshold:
            # Spike occurs immediately; phase reset to zero
            phi_prime = 0
        else:
            # Calculate new phase after perturbation
            phi_prime = phase_from_voltage(V_perturbed, tau, I)

        # Calculate phase change
        delta_phi = phi_prime - phase_from_voltage(V, tau, I)
        delta_phi_values.append(delta_phi)

    return V_values, phi_values, delta_phi_values

# Parameters for perturbation
delta_V = 0.05  # Small perturbation in voltage

# Calculate the PRC
V_values, phi_values, delta_phi_values = calculate_prc(tau, I, delta_V)

# Plot the PRC
plt.figure(figsize=(8, 5))
plt.plot(phi_values / tau, delta_phi_values, label=f'ΔV = {delta_V}')
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.xlabel('Normalized Phase (φ / τ)')
plt.ylabel('Phase Resetting (Δφ)')
plt.title('Phase Resetting Curve (PRC)')
plt.legend()
plt.grid(True)
plt.show()


## Assignment 2b)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# System parameters
def system(t, y):
    R, phi = y
    dR_dt = (1 - R**2) * R  # Radial equation
    dphi_dt = R  # Angular equation
    return [dR_dt, dphi_dt]

# Numerical simulation of the system
def simulate_system(initial_conditions, t_span, dt=0.01):
    t = np.arange(t_span[0], t_span[1], dt)
    sol = solve_ivp(system, t_span, initial_conditions, t_eval=t, method='RK45')
    return sol.t, sol.y

# Step 1: Visualize the limit cycle
t_span = [0, 50]
initial_conditions = [0.5, 0]  # Initial (R, phi) away from the limit cycle
t, y = simulate_system(initial_conditions, t_span)

# Extract results
R, phi = y

# Plot the radial component over time
plt.figure(figsize=(8, 5))
plt.plot(t, R, label="Radius (R)")
plt.axhline(1, color='red', linestyle='--', label="Limit Cycle (R=1)")
plt.xlabel("Time")
plt.ylabel("R")
plt.title("Convergence to Limit Cycle")
plt.legend()
plt.grid()
plt.show()

# Step 2: Compute the asymptotic phase
# Map the phase of the initial conditions onto the limit cycle
def asymptotic_phase(R, phi):
    # Integrate the system to let R approach 1
    t_span = [0, 100]
    sol = solve_ivp(system, t_span, [R, phi], method='RK45')
    R_final, phi_final = sol.y[:, -1]
    return phi_final  # Asymptotic phase on the limit cycle

# Phase calculation example
initial_conditions = [0.8, 0]
asym_phase = asymptotic_phase(initial_conditions[0], initial_conditions[1])
print(f"Asymptotic Phase for initial (R, phi) = {initial_conditions}: {asym_phase}")

# Step 3: Apply a delta pulse and determine PRC
def apply_pulse_and_calculate_prc(delta_R, initial_conditions):
    R, phi = initial_conditions

    # Apply the pulse to R
    R_pulsed = R + delta_R

    # Calculate asymptotic phases before and after the pulse
    asym_phase_original = asymptotic_phase(R, phi)
    asym_phase_pulsed = asymptotic_phase(R_pulsed, phi)

    # PRC: Change in phase
    delta_phi = asym_phase_pulsed - asym_phase_original
    return delta_phi

# Example: Apply a pulse and compute PRC
delta_R = 0.1
initial_conditions = [0.8, 0]
delta_phi = apply_pulse_and_calculate_prc(delta_R, initial_conditions)
print(f"PRC (Δφ) for ΔR = {delta_R}: {delta_phi}")

# Step 4: Plot PRC for a range of initial conditions
initial_R_values = np.linspace(0.5, 1.5, 50)  # Test different initial radii
delta_phi_values = [apply_pulse_and_calculate_prc(delta_R, [R, 0]) for R in initial_R_values]

plt.figure(figsize=(8, 5))
plt.plot(initial_R_values, delta_phi_values, label=f"PRC (ΔR = {delta_R})")
plt.xlabel("Initial Radius (R)")
plt.ylabel("Phase Resetting (Δφ)")
plt.title("Phase Resetting Curve (PRC)")
plt.legend()
plt.grid()
plt.show()
